## Introduction

Here is a list of other things I tried for my Debian AI setup, which didn't work out so well. They didn't cause disasters for me, but they were dead-ends; they didn't solve my setup problems. Here be dragons!

## Allow myself to install packages to /usr/local

Since I wrote this section, Debian has modified their pip so that it refuses to install to `/usr/local`, unless we use the `--break-system-packages` flag.

It's not safe to run `pip install -U` as root. It will merrily remove files from Debian-packaged Python modules under `/usr/lib`, and mess up the system. To avoid this, I changed permissions so that my regular user account can write to everything under `/usr/local` via the `staff` group. I do the same for `/opt` because I want to upgrade Rust from my own user ID also:

In [ ]:
sudo adduser $USER staff
sudo chgrp -R staff /usr/local /opt
sudo chmod -R g+w /usr/local /opt

This is somewhat of a security concern, but if a bad guy gets a local shell it's pretty much game over anyway. To make it safer, I could have used a different non-root account.

In order to upgrade a pip package where the same package was also installed in `/usr/lib` by dpkg, I need to use `pip -I -U packagename`. The `-I` flag tells it to ignore installed packages, i.e. don't try to remove them. I only do that as needed for individual packages, when a normal upgrade fails.

This setup might upset Debian-packaged programs that depend on older versions of the Python libraries that I've upgraded. I haven't noticed any problems like that yet.

If you try to downgrade a package, and if multiple versions of a package are accidentally installed under /usr/local/lib/python3.10 at that same time, it can get confused, so watch out for that. You might need to manually remove one of the versions.

<details>
<summary> Glossary </summary>
    
- pip: A package manager for Python packages that allows users to easily install, upgrade, and remove Python packages.
- /usr/local: A directory on Unix-like systems where locally installed software is usually placed, separate from the operating system's own packages.
- Debian: A popular Linux distribution known for its stability and ease of use.
- --break-system-packages: A flag used with pip to force installation of packages that conflict with system packages, even though this can cause problems.
- root: The default administrative user in Unix-like operating systems, with full permissions to all system files and directories.
- permissions: The settings that control who can access and modify files and directories on a computer.
- staff group: A group on Unix-like systems that provides additional permissions to users who belong to it.
- chgrp: A Unix command used to change the group ownership of a file or directory.
- chmod: A Unix command used to change the permissions of a file or directory.
- security concern: A potential risk to the security of a system, data, or user privacy.
- dpkg: A package manager for Debian-based Linux distributions.
- upgrade: The process of installing a newer version of a software package over an existing version to get new features or bug fixes.
- downgrade: The process of installing an older version of a software package over a newer version.
    
</details>

## Enable switching the system python version

The default Python version for Debian "testing" at time of writing is Python 3.11, however we don't have a stable release of pytorch for Python 3.11 yet.

In [ ]:
python3.11 -m pip install --break-system-packages torchvision

ERROR: Could not find a version that satisfies the requirement torchvision (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3)
ERROR: No matching distribution found for torchvision



: 1

I decided to try switch the system Python version back to 3.10. This is not recommended, but at least it's easy to undo!

Anyway, let's enable switching the system Python version back to 3.10, using `update-alternatives`.

First, note that we can undo this change later, if necessary, as follows:

In [ ]:
sudo update-alternatives --remove-all python3
sudo update-alternatives --remove-all pydoc3
sudo ln -sf python3.11 /usr/bin/python3
sudo ln -sf pydoc3.11 /usr/bin/pydoc3

We can set up alternatives for `python3`:

In [ ]:
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 11
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 10
# sudo update-alternatives --set python3 /usr/bin/python3.10

and do the same for `pydoc3`:

In [ ]:
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.11 11
sudo update-alternatives --install /usr/bin/pydoc3 pydoc3 /usr/bin/pydoc3.10 10
# sudo update-alternatives --set pydoc3 /usr/bin/pydoc3.10

In [ ]:
update-alternatives --list python3

In [ ]:
update-alternatives --list pydoc3

<details>
<summary> Glossary </summary>
    
- torchvision: a PyTorch library containing popular datasets, model architectures, and common image transformations for computer vision.
- update-alternatives: a Debian/Ubuntu tool for managing alternative versions of executables, such as Python or Pydoc.
- system Python: the version of Python installed on a Debian/Ubuntu system and used by default for system-level processes.
- break-system-packages: a pip option that allows installation of packages that may break other system packages.
- ln -s: a command for creating symbolic links between files.
- pydoc: a Python documentation tool.

</details>

## Install nightly torch with Python 3.11

In [ ]:
pip install -qq -U fastbook jupyter jupyterlab tensorflow ipywidgets

At some point I saw a warning about compatibility between fastai and torch.
```
fastai 2.7.11 has requirement torch<1.14,>=1.7, but you have torch 2.0.0.dev20230220+cu117
```
I decided to try using it with nightly torch anyway:

In [ ]:
pip install -qq -U --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu117

Extra packages: nbdev, and kernels for bash and C:

In [ ]:
pip install -qq -U nbdev bash_kernel jupyter-c-kernel

## Use the old pip resolver, if necessary

As of pip 20.3, a new resolver has been introduced, which doesn't always work. As of pip 21.0 the old working resolver is unsupported and slated for removal dependent on pip team resources.

In [ ]:
pip --version

pip 23.0 from /usr/lib/python3/dist-packages/pip (python 3.11)


With this version of pip, it's possible to use the "legacy resolver" with the following option:

In [ ]:
pip install -U -r requirements.txt --use-deprecated=legacy-resolver

Or, I can downgrade pip to version 20.2.4, before they switched to the new resolver.

Refer to https://stackoverflow.com/a/67408694 for more info.

In [ ]:
pip install pip==20.2.4

<details>
<summary> Glossary </summary>
    
- resolver: Part of pip responsible for resolving package dependencies and deciding which versions to install.
- legacy resolver: The older version of pip's resolver algorithm, which is still supported in some versions of pip but has been deprecated and is no longer the default.
  
</details>

## PyPI does not include onnxruntime for python3.11

At one point I was hoping to use python3.11 rather than python3.10. I attempted to build onnxruntime from source, but didn't persevere with it.

<details>
<summary> Glossary </summary>

- PyPI: PyPI (Python Package Index) is a repository of software for the Python programming language. It contains packages of software developed by various authors that are published and shared with the wider Python community.
- onnxruntime: onnxruntime is an open-source engine for executing machine learning models that are defined in the Open Neural Network Exchange (ONNX) format. It provides high performance and supports multiple platforms, including CPU, GPU, and FPGA.
- Source code is the human-readable code that programmers write to create computer programs. It needs to be compiled or interpreted to run on a computer.
- Build: Building refers to the process of compiling or otherwise preparing source code for execution on a specific platform or architecture.
- Open Neural Network Exchange (ONNX): ONNX is an open-source format for representing machine learning models. It is designed to be interoperable across different frameworks and platforms, allowing models to be easily transferred and executed on different hardware.
- FPGA: FPGA stands for Field-Programmable Gate Array. It is an integrated circuit that can be programmed to perform a wide range of computing tasks, including machine learning inference.

</details>

## Untold difficulties! 😭

I haven't described all of the various problems I had to deal with, due to trying various hacky approaches rather than venvs. There were plenty of problems! I guess I learned something from the experience, anyway. I wish Lambda Stack worked on Debian, not only Ubuntu, it would have been a lot easier than this!

<details>
<summary> Glossary </summary>

- hacky approaches: Informal or unconventional methods used to solve a problem or complete a task, often using shortcuts or workarounds instead of following established best practices.
- Lambda Stack: A software stack from Lambda Labs that provides pre-built binaries of deep learning frameworks and other software packages for Ubuntu-based systems.

</details>